# Genetic Improvement Demo

### Create the initial population of Genomes

In [1]:
# system imports
import os, sys
from tqdm import tqdm 


# set the project root directory (~/projects/GIMC)
project_root = os.path.expanduser("~/projects/GIMC")

# add project root to sys.path
sys.path.append(project_root)

# project imports
from genetic_improvement.ollamachat import OllamaChat
from genetic_improvement.config import USER_PROMPT, UNIT_TEST_CODE, SYSTEM_PROMPT, NUM_VARIANTS, BSI_CLASSIFICATION
from genetic_improvement.genome import Genome

# CONSTS
POPULATION_SIZE = 20


In [2]:
# get the initial population from the LLM
population = []
for i in tqdm(range(POPULATION_SIZE // NUM_VARIANTS)):
    ollama_chat = OllamaChat(model="qwen3-coder:latest", system_prompt=SYSTEM_PROMPT, temperature=0.3)
    chat_response = ollama_chat.chat(user_text=USER_PROMPT, stream=False)

    # parse the chat response and add it to the population
    variants = OllamaChat.parse_variants(chat_response)
    candidates = OllamaChat.submit_variants(variants, classification=BSI_CLASSIFICATION)
    for candidate in candidates:
        if candidate is not None:
            genome = Genome(candidate_hash=candidate)
            population.append(genome)

 20%|██        | 1/5 [00:24<01:38, 24.70s/it]

Code submitted successfully. Server response:
Code submitted successfully. Server response:
Code submitted successfully. Server response:
Code submitted successfully. Server response:


 40%|████      | 2/5 [01:00<01:34, 31.46s/it]

Code submitted successfully. Server response:
Code submitted successfully. Server response:
Code submitted successfully. Server response:
Code submitted successfully. Server response:


 60%|██████    | 3/5 [01:28<00:59, 29.74s/it]

Code submitted successfully. Server response:
Code submitted successfully. Server response:
Code submitted successfully. Server response:
Code submitted successfully. Server response:


 80%|████████  | 4/5 [01:59<00:30, 30.17s/it]

Code submitted successfully. Server response:
Code submitted successfully. Server response:
Code submitted successfully. Server response:
Code submitted successfully. Server response:


100%|██████████| 5/5 [02:40<00:00, 32.18s/it]

Code submitted successfully. Server response:
Code submitted successfully. Server response:
Code submitted successfully. Server response:
Code submitted successfully. Server response:


In [5]:
# get the status of each candidate in the population
for genome in population:
    print(f"Candidate {genome.candidate_hash}")


Candidate f410b5bddf4ad9a26c9c5ac4f8971fabd76721a5d818c1279340f135bf1ff32c
Candidate bf0b08e59068bef1f6b485b6944a204e2cf51e8a64e15457ebb63af725cb2114
Candidate f88af3bce6b8859ae1ac5212949cec62c0db6c746fdaaa311390ea381278929b
Candidate 3d0df34679f5500e84948e33baeb59577a4261e212b6df1357bbe9126c6d3c6a
Candidate 1072a2f6e74ffa177ecb5ad7f5740c842e64b45c5f159870228d368864d0b31a
Candidate 7ee2fce9df7c8df2e4edcd9223da33cb66cb2dcd85f67e42a22ca9a21f155525
Candidate 3cacfe0dd7fadcfba225bab0308ddbf836247d1259991ebf909d22803373df1c
Candidate 40022fc797da2bfa1a53391e94dbeb6aa6225fa1280e71c27d60a3f3b37a4eac
Candidate 869691802c9310db29477838b8c563521ce3d59373495026c33a90800af16570
Candidate 96da38f624835c3cfa629fff1d47a898e783a817ae34e158cf0b6f47f603a539
Candidate 2ed4b2553827cb039c004a83ebb89620c700ac3a02d396eee0d6f6f0404f3168
Candidate 059ba64e55c6d24dc50cdc082e4890d04767b07dcea200715e09ce559ffef90e
Candidate 9eeff08a8d10fc1a3a349f34570dbcb511456e3904449dc00a5bd64191ea5c94
Candidate 72f3260a0b195c2